# Data section

## List of Berlin neighborhoods

First of all it is necessary to know which neighborhoods are part of Berlin. <br>
Therefore the following wiki page is scraped https://de.wikipedia.org/wiki/Verwaltungsgliederung_Berlins <br>
In the middle of this page is the following table in german language:
<img src="Berlin neighborhoods.png">

In [ ]:
#import all relevant packages

import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
import time
import numpy as np

#!conda install -c conda-forge folium=0.5.0 --yes
import folium
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [ ]:
res = requests.get("https://de.wikipedia.org/wiki/Verwaltungsgliederung_Berlins")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[2] 
pd.read_html(str(table))